In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 6.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-12_shtle
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-12_shtle
  Resolved https://github.com/openai/CLIP.git to commit d50d76daa670286dd6cacf3bcd80b5e4823fc8e1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369408 sha256=9451fb5fdddd6b836f75b296ef7c2a463c00de4b5d364a4ac1dd02b140de65a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-4cdad1zb/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip


In [2]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)


Torch version: 1.13.1+cu116


In [3]:
#@title Load model 

import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [4]:
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 127MiB/s]


Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan 20 09:00:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    49W / 400W |   1324MiB / 40536MiB |     16%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Image Preprocessing

Resize input images and center-crop them to conform with the image resolution that the model expects. Before doing so, normalize the pixel intensity using the dataset mean and standard deviation



In [6]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7feaad190d30>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

# Setting up input images


In [7]:
from google.colab import drive
from PIL import Image
import os
import cv2
import IPython.display
import matplotlib.pyplot as plt

drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
id_order = list(open("/content/drive/My Drive/DSP/Object_ids.txt"))

id_order = list(map(lambda x:x.strip("\n"), id_order))

In [9]:
#id_order_sample = ['SK-A-2099', 'SK-A-1505', 'SK-A-3841', 'SK-A-476', 'SK-A-3584', 'SK-C-1368', 'SK-A-135', 'SK-A-128']

In [10]:
os.chdir("/content/drive/My Drive/DSP/images")
os.listdir()

['SK-A-3051.jpg',
 'SK-A-3417.jpg',
 'SK-A-2465.jpg',
 'NG-2010-40.jpg',
 'SK-A-3889.jpg',
 'SK-A-955.jpg',
 'SK-A-864.jpg',
 'SK-A-4622.jpg',
 'SK-A-652.jpg',
 'SK-A-3969.jpg',
 'SK-A-1478.jpg',
 'SK-A-177.jpg',
 'SK-A-943.jpg',
 'SK-A-1402.jpg',
 'SK-A-1695.jpg',
 'SK-A-251.jpg',
 'SK-A-148.jpg',
 'SK-A-3127.jpg',
 'SK-A-812.jpg',
 'SK-A-527.jpg',
 'SK-A-535.jpg',
 'SK-A-2845.jpg',
 'SK-A-2042.jpg',
 'SK-A-12.jpg',
 'SK-A-2277.jpg',
 'SK-A-417.jpg',
 'SK-A-885.jpg',
 'SK-A-833.jpg',
 'SK-A-3585.jpg',
 'SK-C-276.jpg',
 'SK-A-3440.jpg',
 'SK-A-734.jpg',
 'SK-A-3113.jpg',
 'SK-A-663.jpg',
 'SK-A-1123.jpg',
 'SK-A-1399.jpg',
 'SK-A-1633.jpg',
 'SK-A-4124.jpg',
 'SK-A-2137.jpg',
 'SK-A-4068.jpg',
 'SK-A-730.jpg',
 'SK-A-514.jpg',
 'SK-A-551.jpg',
 'SK-A-1960.jpg',
 'SK-A-2397.jpg',
 'SK-A-1526.jpg',
 'SK-A-3527.jpg',
 'SK-A-651.jpg',
 'SK-A-4854-C.jpg',
 'SK-A-1352.jpg',
 'SK-A-2329.jpg',
 'SK-A-606.jpg',
 'SK-C-104.jpg',
 'SK-A-977.jpg',
 'SK-A-18.jpg',
 'SK-A-758.jpg',
 'SK-A-204.jpg',


In [11]:
!pwd

/content/drive/My Drive/DSP/images


In [ ]:
original_art_images = []
art_images = []
art_images2 = []
painting_id = []

for filename in os.listdir("/content/drive/My Drive/DSP/images"):
  try:
      painting_id.append(os.path.splitext(filename)[0])
      im = Image.open(filename).convert("RGB")
      art_images.append(preprocess(im))
      art_images2.append(preprocess(im))
      original_art_images.append(im)
  except Exception:
    continue 

In [ ]:
print(len(original_art_images))

In [ ]:
#@title Rearanging painting order

dict_original_art = dict(zip(painting_id, original_art_images))
dict_art = dict(zip(painting_id, art_images))
dict_art2 = dict(zip(painting_id, art_images2))
print(dict_original_art)

In [ ]:
index_map = {v: i for i, v in enumerate(id_order)}
dict_original_art = dict(sorted(dict_original_art.items(), key=lambda pair: index_map[pair[0]]))
dict_art = dict(sorted(dict_art.items(), key=lambda pair: index_map[pair[0]]))
dict_art2 = dict(sorted(dict_art2.items(), key=lambda pair: index_map[pair[0]]))
print(dict_original_art)

In [ ]:
painting_id = list(dict_original_art.keys())
original_art_images = list(dict_original_art.values())
art_images = list(dict_art.values())
art_images2 = list(dict_art2.values())

## Building features

Normalize the images and run the forward pass of the model to get the image features

In [ ]:
art_input = torch.tensor(np.stack(art_images)).cuda()
art_2_input = torch.tensor(np.stack(art_images2)).cuda()

In [ ]:
with torch.no_grad():
    art_features = model.encode_image(art_input).float()
    art_2_features = model.encode_image(art_2_input).float()

## Calculating cosine similarity of image-image pair

Normalize the features and calculate the dot product of pair

In [ ]:
art_features /= art_features.norm(dim=-1, keepdim=True)
art_2_features /= art_2_features.norm(dim=-1, keepdim=True)
similarity = art_2_features.cpu().numpy() @ art_features.cpu().numpy().T

In [ ]:
import pandas as pd

df = pd.DataFrame(similarity)

df.set_axis(painting_id, axis = 0, inplace = True)
df.set_axis(painting_id, axis = 1, inplace = True)

df.head()

In [ ]:
df.to_csv("/content/drive/MyDrive/DSP/clip_similarity.csv")

In [ ]:
#@title Similarity matrix

count = len(art_images)

plt.figure(figsize=(20, 14))
plt.imshow(similarity, vmin=0.4, vmax=1)
# plt.colorbar()
plt.yticks(range(len(art_images)), original_art_images)
plt.xticks([])
for i, image in enumerate(original_art_images):
    plt.imshow(image, extent=(i - 0.5, i + 0.5, -1.6, -0.6), origin="lower")
for x in range(similarity.shape[1]):
    for y in range(similarity.shape[0]):
        plt.text(x, y, f"{similarity[y, x]:.2f}", ha="center", va="center", size=12)

for side in ["left", "top", "right", "bottom"]:
  plt.gca().spines[side].set_visible(False)

plt.xlim([-0.5, count - 0.5])
plt.ylim([count + 0.5, -2])

plt.title("Cosine similarity between visual features", size=20)